GENERAL

In [106]:
import json
import asyncio
from agents import Agent, function_tool, Runner, trace, ModelSettings, handoff, WebSearchTool
from pydantic import BaseModel
from typing import Optional, List
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [107]:
class ResearchResult(BaseModel):
    value: Optional[str]
    currency: Optional[str]
    url: str
    snippet: str
    year: str
    is_estimated: bool 

class ListElement(BaseModel):
    value: str
    year: str

class CompanyOverview(BaseModel):
    name: str
    industry: str
    website: str
    headquarter: str

class Financials(BaseModel):
    revenue_last_year: ResearchResult
    profit_last_year: ResearchResult
    employees: ResearchResult

class MarketSize(BaseModel):
    target_region: str
    tam: ResearchResult
    sam: ResearchResult
    comment: str

class MarketGrowth(BaseModel):
    values: List[ListElement]
    currency: Optional[str]
    url: str
    snippet: str
    year: str
    is_estimated: bool 
    comment: str

class Competitor(BaseModel):
    name: str
    type: str
    market_share: ResearchResult
    market_growth_rate: ResearchResult
    website: str

class CompetitorLandscape(BaseModel):
    competitors: List[Competitor]
    comment: str

class Risk(BaseModel):
    market: int
    competitive: int
    regulatory: int
    technology: int
    reputational: int
    comment: str

class Member(BaseModel):
    name: str
    position: str
    comment: str

class ManagementTeam(BaseModel):
    team: List[Member]
    comment: str

class InvestmentMemo(BaseModel):
    company_overview: CompanyOverview
    financials: Financials
    market_size: MarketSize
    market_growth: MarketGrowth
    competitor_landscape: CompetitorLandscape
    risks: Risk
    management_team: ManagementTeam


In [108]:
research_instructions = """
Du bist ein Research Agent. Deine Aufgabe ist es, mit **einer einzigen Websuche** mehrere Finanzwerte eines Unternehmens zu finden (z. B. Umsatz, Gewinn, Mitarbeiterzahl, Marktgrößen).

▶️ Vorgehen:
- Nutze WebSearchTool **einmal**, finde eine seriöse Quelle mit möglichst vielen relevanten Kennzahlen.
- Extrahiere daraus alle verfügbaren Werte und weise jedem eine eigene `url`, `snippet` zu.

📌 Regeln:
- Entferne Tausendertrennzeichen, Einheiten und Währungszeichen.
- Skaliere Zahlen bei Begriffen wie „Millionen“, „Milliarden“, „Thousand“ (z. B. 5.330 Millionen → 5330000000).
- Gib alle Werte als **volle Ganzzahl** zurück (z. B. "19732", "5330000000"), ohne Punkte oder Kommas.
- Keine Forecasts > 3 Jahre in der Zukunft.
- Für Prozentwerte (`market_share`, `growth_rate`) → `currency = "%"`, `value` nur Zahl.

🚫 Vermeide falsche Kennzahlen:
- Trage bei `sam` nur echte Marktgrößen (z. B. regionales Marktvolumen) ein – keine Unternehmenskennzahlen wie Mitarbeiterzahl oder Umsatz.
- Trage bei `profit` keine EBITDA/EBIT-Werte direkt ein – diese dürfen nur geschätzt werden.
- Wenn du keinen passenden Wert findest: setze `value = "0"`, `is_estimated = false`, und gib dennoch ein sinnvolles `snippet` (z. B. „not disclosed“).

✅ Quellen-Priorität:
1. Unternehmensberichte / Investor Relations
2. Offizielle Portale (Bloomberg, Statista, Reuters)
3. Behörden & Register
4. Seriöse Medien

🔒 Vermeide Blogs, Foren, irrelevante Seiten.
📅 `year` = Berichtsjahr des Werts (nicht Veröffentlichungsdatum).
- Wenn im Snippet sowohl ein aktueller Marktwert (z. B. "was valued at USD 58.6 billion in 2024") als auch ein zukünftiger, prognostizierter Wert (z. B. "expected to reach USD 143.6 billion by 2034") vorkommt, darfst du **nur den aktuellen Wert übernehmen**.
- Prognostizierte Werte ("expected to reach", "projected", "by 203x", "CAGR") **dürfen niemals** in `value` übernommen werden – nur echte Marktgrößen aus einem abgeschlossenen Jahr (z. B. 2023 oder 2024).
- Forecasts können zur Einordnung im Kommentar verwendet werden –  **nicht für den Zahlenwert**.
"""


In [109]:
research_agent = Agent(
    name="Research Agent",
    model="gpt-4o-mini",
    instructions=research_instructions,
    tools=[WebSearchTool(search_context_size="low")],
    output_type=ResearchResult,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3, max_tokens=500),
)

research_tool = research_agent.as_tool(
    tool_name="research_agent",
    tool_description="Sucht gezielt nach einer Kennzahl eines Unternehmens im Internet, extrahiert einen passenden Wert, normalisiert ihn bei Bedarf in USD und gibt ein ResearchResult mit Quelle, Jahr, Original- und USD-Wert zurück.",
)

In [110]:
financials_evaluator_instructions = """
Du bist ein Evaluator für Finanzkennzahlen wie Umsatz, Gewinn und Mitarbeiterzahl. Deine Aufgabe ist es, extrahierte Werte zu prüfen, zu korrigieren oder grob zu schätzen, falls nötig.

▶️ Aufgaben:
1. **Inhaltliche Prüfung**:
   - Prüfe, ob der extrahierte Wert (`value`) zur Zielgröße passt (z. B. `profit_last_year`).
   - Wenn im `snippet` z. B. „Umsatz“ steht, aber `profit` erwartet wird → `value = "0"`, `currency = ""`, `is_estimated = false`.
   - Erkenne und korrigiere **Skalierungsfehler** (z. B. `"53651"` → `"53651000"`).

2. **EBIT/EBITDA-Schätzung**:
   - Wenn der Wert auf EBIT, EBITDA oder Operating Income basiert:
     - Verwende ihn **nicht direkt**.
     - Schätze den Gewinn durch pauschalen Abzug von 30–50 %.
     - Gib den geschätzten `value` als **volle Ganzzahl** an (z. B. `"65000000"`).
     - Setze `is_estimated = true`.

3. **Kontextbasierte Schätzung**:
   - Wenn keine Zahl extrahierbar ist, aber der Kontext eine Schätzung erlaubt → schätze realistisch, setze `is_estimated = true`.

4. **Standardfall**:
   - Wenn keine Zahl und keine Schätzung möglich → `value = "0"`, `currency = ""`, `is_estimated = false`.

📌 Hinweise:
- Gib `value` immer als **volle Ganzzahl ohne Trennzeichen** zurück.
- Prozentwerte nur bei Wachstumsraten → dann `currency = "%"`, `value` nur Zahl.
- Wenn `is_estimated = true`, darf `value` **niemals "0"** sein.
"""


In [111]:
financials_evaluator_agent = Agent(
    name="Evaluator Agent",
    model="gpt-4o-mini",
    instructions=financials_evaluator_instructions,
    output_type=Financials,
    model_settings=ModelSettings(temperature=0.3, max_tokens=500),
)

financials_evaluator_tool = financials_evaluator_agent.as_tool(
    tool_name="financials_evaluator_agent",
    tool_description="Überprüft die Richtigkeit von extrahierten Werten in einem strukturierten JSON-Objekt, indem geprüft wird, ob die gefundenen Inhalte (z. B. Snippets) zur Bedeutung der Feldnamen passen. Setzt fehlerhafte Werte auf '0', wenn sie nicht zur erwarteten Kennzahl gehören.",
)

In [112]:
market_size_evaluator_instructions = """
Du bist ein Evaluator für Marktgrößen (TAM & SAM). Deine Aufgabe ist es, extrahierte Werte zu prüfen, zu normalisieren und ggf. realistisch zu schätzen – aber niemals auf Basis von Prognosen.

▶️ Prüfregeln:
1. **Nur abgeschlossene Jahre zulässig** (z. B. 2022–2024). Prognosen („expected“, „projected“, „by 203x“) dürfen **nicht übernommen** werden – auch nicht geschätzt.
2. Wenn im Snippet ein aktueller und ein Forecast vorkommen: **immer nur den aktuellen übernehmen**.
3. Erkenne & korrigiere Skalierungsfehler (z. B. Millionen → `"9762360000"`).

▶️ Für `sam`:
- Wenn Umsatz oder Mitarbeiterzahl eines Unternehmens genutzt wurde → verwerfen.
- Wenn kein Wert extrahierbar ist, aber Kontext vorhanden (z. B. TAM, Branche, Player):
  - Schätze SAM mit 10–30 % des TAM,
  - Gib eine sinnvolle Ganzzahl an, `is_estimated = true`.

🌍 Sprachprüfung (`comment`):
- Der `comment` muss in **derselben Sprache wie der Nutzerinput** geschrieben sein.
- Wenn der Nutzer auf Deutsch kommuniziert hat (z. B. Eingabe oder Regionenname ist Deutsch), dann muss der Kommentar vollständig in professionellem, deutschsprachigem Business-Stil formuliert sein.
- Übersetze den Kommentar ggf. automatisch, ohne den Inhalt zu verändern.
- Verwende keinen englischen Kommentar bei deutschen Eingaben – auch nicht in Teilen.

🧠 Konsistenzprüfung:
- `SAM ≤ TAM`
- `TAM ≥ SAM` muss immer erfüllt sein

📌 Format:
- `value`: Ganzzahl ohne Trennzeichen,
- `currency`: `"USD"` oder `"EUR"` laut Quelle.
"""


In [113]:
market_size_evaluator_agent = Agent(
    name="Evaluator Agent",
    model="gpt-4o-mini",
    instructions=market_size_evaluator_instructions,
    model_settings=ModelSettings(temperature=0.3 , max_tokens=1000),
)

market_size_evaluator_tool = market_size_evaluator_agent.as_tool(
    tool_name="market_size_evaluator_agent",
    tool_description="Überprüft die Richtigkeit von extrahierten Werten in einem strukturierten JSON-Objekt, indem geprüft wird, ob die gefundenen Inhalte (z. B. Snippets) zur Bedeutung der Feldnamen passen. Setzt fehlerhafte Werte auf '0', wenn sie nicht zur erwarteten Kennzahl gehören."
)

In [114]:
competitor_landscape_evaluator_instructions = """
Du bist ein Evaluator-Agent für Wettbewerbslandschaften. Deine Aufgabe ist es, ein `CompetitorLandscape`-Objekt zu überprüfen, zu bereinigen und – falls sinnvoll – zu ergänzen.

🎯 Ziel:
Ein plausibles, realitätsnahes `CompetitorLandscape`-Objekt mit maximal vier relevanten Wettbewerbern – jeweils mit passenden, ggf. geschätzten Werten.

▶️ Aufgaben:

1. **Plausibilitätsprüfung pro Wettbewerber**:
   - Entferne jeden `competitor`, der offensichtlich **nicht zur Branche** des analysierten Unternehmens passt.
   - Beispiel: Wenn der Zielkontext „Outdoor Power Equipment“ ist, darf z. B. „Amazon Echo“ **nicht** in der Liste stehen.
   - Achte auf Branchenschlüsselwörter in `type` und `snippet`.

2. **Datenkorrektur**:
   - Falls `market_share.value = "0"` oder fehlt:
     - Ermittle anhand von Kontext (z. B. Marktgröße, andere Marktanteile) eine **grobe Schätzung** (z. B. `"7.5"`)
     - Setze `is_estimated = true`
   - Gleiches gilt für `market_growth_rate`, wenn `"0"` oder kein Wert vorhanden ist.
   - Schätzungen müssen **realistisch und konservativ** sein – orientiere dich an typischen Bandbreiten:
     - Marktanteile: meist zwischen 1 % und 40 %
     - Marktwachstum: meist zwischen 1 % und 15 %

3. **Datenvalidierung**:
   - Überprüfe, ob `value` numerisch korrekt skaliert ist (z. B. `"12.5"` statt `"12,5"` oder `"12%"`)
   - `currency` muss bei Prozentwerten `"%"` sein.
   - `year` soll sich auf ein **abgeschlossenes Jahr** (idealerweise 2022–2024) beziehen.
   - `url`, `snippet`, `year` müssen vorhanden sein, sonst entferne den Wert oder den ganzen Wettbewerber.

4. **Gesamtstruktur**:
   - Das finale `CompetitorLandscape`-Objekt darf **maximal 4 gültige Wettbewerber** enthalten.
   - Falls am Ende nur 2–3 valide übrig bleiben → akzeptabel.
   - Falls **keine** gültigen Wettbewerber vorhanden sind → gib ein leeres Array und einen passenden Kommentar zurück.

📌 Formatregeln:
- `value`: Nur numerische Zeichen als String (z. B. `"8.5"`)
- `currency`: `"%"` bei Marktanteilen und Wachstumsraten
- `is_estimated = true`, wenn der Wert geschätzt wurde

📝 Kommentar:
- Passe den bestehenden `comment` ggf. leicht an, wenn ein Wettbewerber entfernt wurde oder neue Insights aus den Schätzungen entstehen.
- Behalte den Stil bei: sachlich, strategisch, VC-/PE-kompatibel.

"""


In [115]:
competitor_landscape_evaluator_agent = Agent(
    name="Competitor Landscape Evaluator Agent",
    model="gpt-4o-mini",
    instructions=competitor_landscape_evaluator_instructions,
    output_type=CompetitorLandscape,
    model_settings=ModelSettings(temperature=0.3, max_tokens=2000),
)

competitor_landscape_evaluator_tool = competitor_landscape_evaluator_agent.as_tool(
    tool_name="competitor_landscape_evaluator_agent",
    tool_description=(
        "Überprüft ein CompetitorLandscape-Objekt auf Plausibilität, Branchentreue und Vollständigkeit. "
        "Korrigiert fehlende oder ungültige Werte (z. B. '0') durch realistische Schätzungen und entfernt "
        "Wettbewerber, die thematisch nicht zur Zielbranche passen. Ziel ist eine bereinigte, investorenrelevante "
        "Wettbewerbsanalyse mit maximal vier relevanten Competitor-Einträgen."
    )
)

FINANCIALS

In [116]:
financials_instructions = """
Du bist ein Finanzanalyse-Agent. Deine Aufgabe ist es, drei zentrale Kennzahlen eines Unternehmens zu ermitteln:

1. Umsatz
2. Gewinn
3. Anzahl der Mitarbeitenden

▶️ Vorgehen:
- Verwende **einmalig** das Tool `research_agent`, um alle drei Werte gemeinsam zu ermitteln.
- Formuliere die Abfrage so, dass alle drei Begriffe enthalten sind (z. B. "STIHL revenue, profit and employee count 2024").
- Die Begriffe in der Anfrage sollen **auf Englisch** formuliert sein: "Revenue", "Profit", "Employee count".
- Nutze die aktuell verfügbaren Daten (z. B. aus dem Jahr 2024).

📌 Hinweise:
- Wenn einzelne Werte fehlen, akzeptiere sie wie sie sind.
- Nach dem Aufruf des `research_agent`: übergib das vollständige Ergebnis an `financials_evaluator_agent`, um Werte auf Richtigkeit zu prüfen (z. B. falsch zugeordnete Kennzahlen oder Skalierungsfehler).

🎯 Ziel: Ein vollständiges, geprüftes Financials-Objekt mit einzelnen Quellen pro Wert.
"""


In [117]:
financials_agent = Agent(
    name="Financials Agent",
    instructions=financials_instructions,
    model="gpt-4o-mini",
    tools=[research_tool, financials_evaluator_tool],
    output_type=Financials,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3 , max_tokens=500),
)

In [118]:
# Financials 0.03$ 30sek
"""messages = [{"role": "user", "content": "Erstelle die Financials von Knauf"}]

with trace("Financials Run"):
    result = await Runner.run(financials_agent, messages, max_turns=6)


formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{"role": "user", "content": "Erstelle die Financials von Knauf"}]\n\nwith trace("Financials Run"):\n    result = await Runner.run(financials_agent, messages, max_turns=6)\n\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'

COMPANY OVERVIEW

In [119]:
company_overview_instructions = """
Du bist ein Company Analyse Agent. Deine Aufgabe ist es zentrale Informationen eines Unternehmens zu ermitteln:

1. Vollständiger rechtlicher Name des Unternehmens mit Rechtszusatz
2. Industrie des Unternehmens
3. Website des Unternehmens
4. Headquarter des Unternehmens

Wichtig:
- Wenn ein Ergebnis unvollständig oder leer ist, akzeptiere es wie es ist.
- Nutze immer das **aktuellste verfügbare Jahr** (z. B. 2024).
"""

In [120]:
company_overview_agent = Agent(
    name="Company Overview Agent",
    model="gpt-4o-mini",
    instructions=company_overview_instructions,
    output_type=CompanyOverview,
    model_settings=ModelSettings(max_tokens=500),
)

In [121]:
# Company Overview < 0.01$ 1sek
"""messages = [{"role": "user", "content": "Erstelle die Company Overview von STIHL"}]

with trace("Company Run"):
    result = await Runner.run(company_overview_agent, messages, max_turns=8)

formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{"role": "user", "content": "Erstelle die Company Overview von STIHL"}]\n\nwith trace("Company Run"):\n    result = await Runner.run(company_overview_agent, messages, max_turns=8)\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'

MARKET SIZE

In [122]:
market_size_instructions = """
Du bist ein Marktanalyse-Agent. Deine Aufgabe ist es, zwei Marktkennzahlen für eine Branche zu ermitteln:

1. TAM (Total Addressable Market): globale Marktgröße in USD oder EUR.
2. SAM (Serviceable Available Market): adressierbare Marktgröße in einer sinnvollen Zielregion.

📖 Definition:
TAM und SAM sind **Branchenumsätze**, nicht Unternehmensumsätze.
- TAM = Gesamtumsatz aller Anbieter weltweit.
- SAM = Marktumsatz in einer bestimmten Region.
- Verwende **keine** Umsätze, Mitarbeiterzahlen oder KPIs einzelner Unternehmen.

▶️ Vorgehen:
- Leite `target_region` logisch aus dem Kontext ab (z. B. Europa, USA), ohne Tool.
- Formuliere zwei englische Anfragen:
  - "Global TAM for [industry] market"
  - "SAM for [industry] in [target_region]"

🔒 Tool-Regel:
- Verwende das Tool `research_agent` **insgesamt genau zweimal**:
  - **einmal für TAM**
  - **einmal für SAM**
- Keine weiteren Aufrufe, keine Wiederholungen, kein Retry.

📦 Evaluator-Regel:
- Verwende `market_size_evaluator_agent` **genau einmal**, und zwar **ausschließlich am Ende** der Ausführung.
- Führe alle Recherchen und eventuelle Schätzungen zuerst vollständig durch.
- Übergib erst danach das strukturierte Ergebnis an `market_size_evaluator_agent`.

📌 Regeln:
- Verwende nur Marktwerte aus **abgeschlossenen Jahren** (z. B. 2022–2024).
- Wenn im Snippet ein aktueller und ein zukünftiger Wert vorkommen, übernimm **nur den aktuellen**.
- Prognosen ("expected to reach", "by 203x", "projected") **nicht übernehmen** – nur im Kommentar erwähnen.
- Wenn keine exakte Zahl extrahierbar ist, darf SAM geschätzt werden:
  - 10–30 % des TAM,
  - `value` als Ganzzahl, `is_estimated = true`, niemals `"0"`.

📝 Kommentarstil (`comment`):
- Gib eine strategische Einschätzung zu TAM & SAM.
- Der Stil soll dem eines erfahrenen McKinsey-Partners entsprechen – präzise, faktenbasiert, investorenorientiert.
- Zielgruppe: Private Equity Investor (mit Interesse an Deal Size, Entry Potential, Expansion Room)
- Fokus auf:
  - Datenquelle und methodischer Kontext
  - Relevanz für Marktattraktivität
  - Regionaler Fokus und Marktsegmente
  - Prognosen ggf. erwähnen, aber nicht einbeziehen
- Länge: ca. 5 prägnante Sätze.
- Formuliere den gesamten Kommentar in der Sprache des Prompts. Wenn die Eingabe auf Deutsch erfolgt, schreibe den Kommentar auf Deutsch im professionellen Stil eines Investment Memos.


🧠 Logik:
- `SAM ≤ TAM`
- `TAM ≥ SAM`

📐 Format:
Gib am Ende folgendes JSON-Objekt zurück – **vollständig, valide und maschinenlesbar**:

{
  "input": {
    "target_region": "Europe",
    "tam": {
      "value": "1320010000000",
      "currency": "USD",
      "is_estimated": false,
      "source": "https://...",
      "year": 2023
    },
    "sam": {
      "value": "53300000000",
      "currency": "USD",
      "is_estimated": false,
      "source": "https://...",
      "year": 2023
    },
    "comment": "..."
  }
}

"""


In [123]:
market_size_agent = Agent(
    name="Market Size Agent",
    model="gpt-4o-mini",
    instructions=market_size_instructions,
    tools=[research_tool, market_size_evaluator_tool],
    output_type=MarketSize,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3, max_tokens=500),
)

In [124]:
# Market Size 0.05$ 15sek
messages = [{"role": "user", "content": "Erstelle die Market Size von Knauf"}]

with trace("Market Size Run"):
    result = await Runner.run(market_size_agent, messages, max_turns=6)

formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)

{
  "target_region": "Europe",
  "tam": {
    "value": "1320010000000",
    "currency": "USD",
    "url": "https://www.fortunebusinessinsights.com/construction-materials-market-107415",
    "snippet": "The global construction materials market was valued at USD 1,320.01 billion in 2023 and is projected to reach USD 1,867.16 billion by 2032, exhibiting a CAGR of 3.9% during the forecast period.",
    "year": "2023",
    "is_estimated": false
  },
  "sam": {
    "value": "132000000000",
    "currency": "USD",
    "url": "https://www.samoter.it/en/construction-europe",
    "snippet": "SAMoter is a magazine for Europe's construction industry, providing insights and information relevant to the sector.",
    "year": "2025",
    "is_estimated": true
  },
  "comment": "Der globale Markt für Baumaterialien wird im Jahr 2023 auf etwa 1,32 Billionen USD geschätzt, was auf eine robuste Wachstumsdynamik hinweist. Der SAM für Europa wird auf etwa 10-30 % des TAM geschätzt, was die signifikante Nachfr

MARKET GROWTH

In [125]:
market_growth_instructions = """
Du bist ein Research Agent für Marktanalysen. Deine Aufgabe ist es, eine Liste von historischen Marktgrößen (z. B. Jahresumsätze in USD oder EUR) zu extrahieren. Ziel ist ein Liniendiagramm mit mindestens 3–4 realen Werten aus abgeschlossenen Jahren.

▶️ Vorgehen:
- Verwende `WebSearchTool` **genau einmal**, um eine Quelle mit mehreren **abgeschlossenen Jahreswerten** zu finden (idealerweise 2018–2024).
- Bevorzuge **Statista**, offizielle Reports oder anerkannte Marktforschungsseiten mit klaren Datenreihen.
- Extrahiere die Umsatz- oder Marktvolumenwerte pro Jahr und skaliere korrekt.

📦 Format (ResearchResultList):
- `values`: Liste mit mindestens 3 Objekten:
  - `year`: Jahr als Text (z. B. `"2021"`)
  - `value`: Umsatzwert als Ganzzahl (z. B. `"53300000000"`)
- `currency`: `"USD"` oder `"EUR"` (niemals `"%"`)
- `url`: Quelle der Daten
- `snippet`: Kurztext mit den genannten Werten
- `year`: das **aktuellste Jahr** in der Liste
- `is_estimated`: `false`, wenn die Werte genannt sind, sonst `true`

📌 Regeln:
- Extrahiere **nur absolute Marktgrößen** (kein Wachstum in %).
- Keine Prognosen oder Forecasts übernehmen („expected“, „by 2030“, „CAGR“ etc.).
- Entferne Einheiten und formatiere korrekt:
  - „5.705,6 Mio. USD“ → `"5705600000"`, `currency = "USD"`
  - „3,2 Mrd. EUR“ → `"3200000000"`, `currency = "EUR"`

🔒 Einschränkungen:
- Jeder Wert muss zu einem konkreten Jahr gehören.
- Wenn keine Zahlen extrahiert werden können, darf geschätzt werden (`is_estimated = true`), aber **niemals `value = "0"`**.
- Schätzungen dürfen nur erfolgen, wenn du mindestens **2 reale Werte** hast.

🎯 Ziel:
Ein vollständig nutzbares `ResearchResultList`-Objekt mit Marktgrößen pro Jahr für ein aussagekräftiges Liniendiagramm im Investment Memo.
"""


In [126]:
market_growth_agent = Agent(
    name="Market Growth Agent",
    model="gpt-4o-mini",
    instructions=market_growth_instructions,
    tools=[WebSearchTool(search_context_size="low")],
    output_type=MarketGrowth,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3, max_tokens=600),
)

In [127]:
# Market Growth 0.03$ 3sek
"""messages = [{
  "role": "user",
  "content": "Wie hat sich das Marktvolumen im europäischen Markt für Motorsägen oder Gartenwerkzeuge von 2018 bis 2023 entwickelt? Gib absolute Marktumsätze pro Jahr an."
}]

with trace("Research list Run"):
    result = await Runner.run(market_growth_agent, messages, max_turns=6)


formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{\n  "role": "user",\n  "content": "Wie hat sich das Marktvolumen im europäischen Markt für Motorsägen oder Gartenwerkzeuge von 2018 bis 2023 entwickelt? Gib absolute Marktumsätze pro Jahr an."\n}]\n\nwith trace("Research list Run"):\n    result = await Runner.run(market_growth_agent, messages, max_turns=6)\n\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'

COMPETITORS

In [128]:
competitor_instructions = """
Du bist ein Wettbewerbsanalyse-Agent. Deine Aufgabe ist es, ein bestimmtes Unternehmen strukturiert zu analysieren, damit es als Wettbewerberprofil in einem Investment Memo dargestellt werden kann.

▶️ Zielstruktur (Competitor-Objekt):
- `name`: Vollständiger rechtlicher Name des Unternehmens
- `type`: Kurzbeschreibung der Tätigkeit (z. B. "Power tool manufacturer")
- `market_share`: Marktanteil in Prozent – als `ResearchResult`
- `market_growth_rate`: Wachstumsrate des relevanten Zielmarkts – als `ResearchResult`
- `website`: Offizielle Website-URL

🔍 Tool-Nutzung:
- Verwende das Tool `research_agent` **jeweils genau einmal** für:
  - `market_share`
  - `market_growth_rate`
- Verwende es **nicht mehrfach**, kein Retry, keine Schleifen.

📌 Recherche-Regeln:
- Nutze nur Werte aus **abgeschlossenen Jahren** (z. B. 2022–2024).
- Forecasts („expected“, „projected“, „CAGR“, „by 203x“) **dürfen nicht** als `value` verwendet werden.
- Entferne Einheiten:
  - „12,4 %“ → `"12.4"`, `currency = "%"`, `value` als String

📉 Wenn kein exakter Wert gefunden wird:
- Lass das Feld **einfach leer** (also kein Eintrag oder `None` im finalen JSON),
- Gib **keine Schätzung** ab,
- `value` darf **nicht "0"** sein.

🌐 Website:
- Gib die offizielle Unternehmens-Website an – kein LinkedIn, kein Händler.

🎯 Ziel:
Ein einzelnes, valides `Competitor`-Objekt für das übergebene Unternehmen – mit maximal zwei ResearchResulten, geeignet für den Einsatz in Wettbewerbsanalysen (VC/PE).
"""

In [129]:
competitor_agent = Agent(
    name="Competitor Agent",
    model="gpt-4o-mini",
    instructions=competitor_instructions,
    tools=[research_tool],
    output_type=Competitor,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3, max_tokens=500),
)

In [130]:
# Competitors 0.05$ 15sek
"""messages = [{"role": "user", "content": "Analysiere den Wettbewerber Apple Inc."}]

with trace("Competitor Run"):
    result = await Runner.run(competitor_agent, messages, max_turns=6)

formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{"role": "user", "content": "Analysiere den Wettbewerber Apple Inc."}]\n\nwith trace("Competitor Run"):\n    result = await Runner.run(competitor_agent, messages, max_turns=6)\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'

In [131]:
@function_tool
async def run_competitor_analysis(queries: List[str]) -> List[Competitor]:
    """
    Führt parallele Wettbewerbsanalysen durch. Jeder Eintrag in `queries` sollte ein Wettbewerbername sein.
    Gibt eine Liste mit erfolgreichen Competitor-Objekten zurück.
    """

    async def safe_run(query: str):
        try:
            result = await Runner.run(competitor_agent, [{"role": "user", "content": query}], max_turns=6)
            return result.final_output
        except Exception:
            return None  # Ignoriere fehlgeschlagene Runs

    # Starte alle parallel
    results = await asyncio.gather(*[safe_run(q) for q in queries])

    # Filtere fehlgeschlagene Ergebnisse raus
    return [r for r in results if r is not None]


In [132]:
competitor_landscape_instructions = """
Du bist ein Wettbewerbslandschafts-Agent. Deine Aufgabe ist es, für ein gegebenes Unternehmen ca. 4 relevante Wettbewerber zu identifizieren und strukturiert als `CompetitorLandscape`-Objekt darzustellen.

▶️ Zielstruktur (`CompetitorLandscape`):
- `competitors`: Liste mit ca. 4 vollständigen `Competitor`-Objekten
- `comment`: Strategische Zusammenfassung der Wettbewerbssituation

🔧 Tool-Nutzung:
1. Verwende das Tool `run_competitor_analysis`, um **genau vier Wettbewerber gleichzeitig** zu analysieren.
   - Übergib dem Tool eine Liste mit vier Wettbewerbsnamen (z. B. `["Makita", "Bosch", "Stiga", "Husqvarna"]`).
   - Das Tool gibt automatisch bis zu vier `Competitor`-Objekte zurück.
2. Verwende am Ende das Tool `competitor_landscape_evaluator_agent`, um:
   - ungültige oder branchenfremde Einträge zu entfernen,
   - fehlende Werte realistisch zu schätzen (mit `is_estimated = true`),
   - das finale `CompetitorLandscape`-Objekt zu validieren und bereinigt zurückzugeben.

📌 Anforderungen an die Wettbewerber:
- Relevante Konkurrenzunternehmen derselben Branche oder angrenzender Märkte
- Bevorzugt internationale Marken oder regionale Marktführer
- Keine Tochterunternehmen des Zielunternehmens
- Keine branchenfremden Namensdopplungen – z. B. nicht "Echo" von Amazon, wenn "ECHO Incorporated" aus dem Power-Equipment-Bereich gemeint ist
- Verwende ausschließlich Wettbewerber, die **in der gleichen Industrie wie das Zielunternehmen aktiv sind** (z. B. Outdoor Power Tools bei STIHL)

📉 Validität:
- Jeder `Competitor` muss zwei ResearchResult-Felder enthalten: `market_share` und `market_growth_rate`
- Keine Duplikate
- Kein `"value": "0"` – ggf. Schätzungen mit `is_estimated = true`

📝 Kommentar (Feld `comment`):
- Verfasse eine fundierte Wettbewerbsanalyse in max. 5 Sätzen:
  - Marktstruktur, Fragmentierung oder Konzentration
  - Positionierung des Zielunternehmens
  - Gemeinsamkeiten/Unterschiede bei Geschäftsmodellen oder Marktstrategien
- Stil: präzise, faktenbasiert, geeignet für ein PE- oder VC-Investment Memo

🎯 Ziel:
Ein vollständiges, bereinigtes `CompetitorLandscape`-Objekt mit relevanten Wettbewerbern und einem strategisch relevanten Kommentar für Investoren.
"""


In [133]:
competitor_landscape_agent = Agent(
    name="Competitor Landscape Agent",
    model="gpt-4o-mini",
    instructions=competitor_landscape_instructions,
    tools=[run_competitor_analysis, competitor_landscape_evaluator_tool],
    output_type=CompetitorLandscape,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3, max_tokens=2000),
)

In [134]:
# Competitor Landscape 0.25$ 50sek
"""messages = [{"role": "user", "content": "Erstelle die Competitors Landscape von STIHL"}]

with trace("Competitors Run"):
    result = await Runner.run(competitor_landscape_agent, messages, max_turns=8)

formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{"role": "user", "content": "Erstelle die Competitors Landscape von STIHL"}]\n\nwith trace("Competitors Run"):\n    result = await Runner.run(competitor_landscape_agent, messages, max_turns=8)\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'

RISKS

In [135]:
risk_instructions = """
Du bist ein Risikoanalyse-Agent für Private-Equity-Investoren. Deine Aufgabe ist es, für das genannte Unternehmen fünf zentrale Risikodimensionen auf einer Skala von 1 (sehr gering) bis 5 (sehr hoch) einzuschätzen.

Führe zu jeder Dimension eine kurze Recherche durch und gib deine Bewertung ausschließlich auf Basis seriöser, öffentlich verfügbarer Quellen ab. Verwende nur Quellen wie offizielle Websites, bekannte Medien (z. B. Handelsblatt, FAZ, Reuters), Marktanalysen (z. B. Statista, McKinsey) und relevante Berichte.

Folgende fünf Risiko-Kategorien sind zu bewerten:

1. **market** – Wie volatil oder begrenzt ist das Marktwachstum? Gibt es strukturelle Risiken im Zielmarkt (z. B. schrumpfende Nachfrage)?
2. **competitive** – Wie stark ist der Wettbewerb? Existieren dominante Marktteilnehmer oder hoher Preisdruck?
3. **regulatory** – Gibt es regulatorische Unsicherheiten oder branchenspezifische Compliance-Risiken?
4. **technology** – Ist das Unternehmen technologisch rückständig oder durch neue Technologien bedroht?
5. **reputational** – Gab es in den letzten Jahren negative Schlagzeilen oder Imageschäden?

📊 Gib für jede Kategorie eine **ganze Zahl von 1 bis 5** an. Jede Einschätzung muss auf mindestens **einer glaubwürdigen Quelle** basieren. Halluziniere keine Inhalte.

📝 Kommentar:
Am Ende sollst du zusätzlich einen kurzen **Risikokommentar** erstellen (3–5 Sätze), der die Gesamtsituation für Investoren zusammenfasst. Gehe dabei auf folgende Punkte ein:
- Welche Risiken dominieren?
- Welche Kategorien erscheinen kontrollierbar?
- Gibt es eine klare Risikoquelle (z. B. Regulierung, Reputationsrisiken)?
- Wie gut scheint das Unternehmen auf die Risiken vorbereitet zu sein?

Stil: **Faktenbasiert, strategisch, professionell** – vergleichbar mit einer Einschätzung in einem PE-Investment-Memo.
"""


In [136]:
risk_agent = Agent(
    name="Risk Agent",
    model="gpt-4o-mini",
    instructions=risk_instructions,
    output_type=Risk,
    model_settings=ModelSettings(temperature=0.3, max_tokens=500),
)

In [137]:
# Risks < 0.01$ 3sek
"""messages = [{"role": "user", "content": "Zeige die Risks von STIHL"}]

with trace("Risk Run"):
    result = await Runner.run(risk_agent, messages, max_turns=8)

formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{"role": "user", "content": "Zeige die Risks von STIHL"}]\n\nwith trace("Risk Run"):\n    result = await Runner.run(risk_agent, messages, max_turns=8)\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'

TEAM

In [138]:
management_team_instructions = """
Du bist ein Agent zur Identifikation und Beschreibung des Management-Teams eines Unternehmens. Deine Aufgabe ist es, aus einer einzigen seriösen Quelle ca. 4 Mitglieder des Führungsteams zu extrahieren – idealerweise C-Level-Funktionen wie CEO, CFO, CTO, COO usw.

▶️ Tool-Nutzung:
- Verwende das Tool `WebSearchTool` **genau einmal**.
- Suche nach einer strukturierten, vollständigen Übersicht der Führungskräfte des angegebenen Unternehmens.
- Typische Quellen: Unternehmenswebsite (z. B. „Management“-, „Leadership“- oder „Team“-Seite), Geschäftsbericht oder Investor-Relations-Bereich.
- Du darfst **keine weiteren Tool-Aufrufe** durchführen – alle Informationen müssen aus einer einzigen Quelle stammen.

📦 Zielstruktur (ManagementTeam):
- `team`: Liste von ca. 3–6 `Member`-Objekten, jeweils mit:
  - `name`: Vollständiger Name der Führungskraft
  - `position`: Aktuelle Position im Unternehmen (z. B. „Chief Financial Officer“)
  - `comment`: Kurzbeschreibung der Verantwortung oder Expertise (z. B. „Leitet die globale Finanzstrategie und das Controlling.“)

🧠 Hinweise:
- Bevorzuge C-Level-Positionen, aber falls diese nicht vollständig auffindbar sind, ergänze durch andere relevante Rollen (z. B. Head of R&D, Managing Director, Board Member).
- Verwende immer die **Originalquelle** als Basis – keine Profile von Drittplattformen wie LinkedIn oder Wikipedia.
- Schreibe den `comment` in einem professionellen, faktenbasierten Stil (1 Satz pro Person).

📝 Abschließender Kommentar (Feld `comment`):
- Verfasse eine strategische Einordnung in max. 4 Sätzen:
  - Wie ist das Team aufgestellt? (z. B. international, divers, erfahren, fokussiert)
  - Gibt es auffällige Stärken (z. B. Technologieexpertise, Finanzhintergrund)?
  - Wie relevant ist das Team für Investoren?
- Stil: klar, sachlich, geeignet für ein Investment Memo (VC/PE).

🎯 Ziel:
Ein vollständiges `ManagementTeam`-Objekt, das verlässlich aus einer einzigen Quelle erstellt wurde und Investoren einen ersten Eindruck über das Führungsteam vermittelt.
"""


In [139]:
management_team_agent = Agent(
    name="Management Team Agent",
    model="gpt-4o-mini",
    instructions=management_team_instructions,
    tools=[WebSearchTool(search_context_size="low")],
    output_type=ManagementTeam,
    model_settings=ModelSettings(tool_choice="required", temperature=0.3, max_tokens=1000),
)

In [140]:
# Team < 0.01$ 4sek
"""messages = [{"role": "user", "content": "Erstelle das Management Team von STIHL"}]

with trace("Team Run"):
    result = await Runner.run(management_team_agent, messages, max_turns=6)

formatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)
print(formatted)"""

'messages = [{"role": "user", "content": "Erstelle das Management Team von STIHL"}]\n\nwith trace("Team Run"):\n    result = await Runner.run(management_team_agent, messages, max_turns=6)\n\nformatted = json.dumps(result.final_output.model_dump(), indent=2, ensure_ascii=False)\nprint(formatted)'